In [1]:
# (ignore this) imports and utility functions

import re
import json
import toolkit

from textwrap import dedent
from IPython.display import clear_output, display, Markdown


def render_markdown(text):
    if text is None:
        text = ""

    pattern = r"```thinking\n(.*?)\n```"

    def replacement(match):
        content = match.group(1)
        return f"<blockquote>\n{content}\n</blockquote>\n\n"

    text = re.sub(pattern, replacement, text, flags=re.DOTALL)

    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    display(Markdown(text))


def render_json(data):
    return render_markdown(f"```json\n{json.dumps(data, indent=2)}\n```\n")


def render_live(stream):
    response = ""
    render_markdown("> Loading...")

    for chunk in stream:
        response += chunk
        clear_output(wait=True)
        render_markdown(response)

    return response

In [2]:
model = "gemma3:27b"

In [3]:
await toolkit.code.create_sandbox()

In [4]:
prompt = toolkit.prompts.function_writing(model)
task = dedent("""
    I'd like to be able to ask you for the weather. Could you please write code to use the wttr.in API to fetch the weather for me
    when I need it? Just make a request to this URL: 'http://wttr.in/<place>?format=(%l) %C and %t (feels %f) with winds at %w'.
""")

In [5]:
messages = [{"role": "user", "content": prompt}, {"role": "user", "content": task}]

In [6]:
streaming_response = toolkit.model.get_response(model, messages)
function_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": function_response})

```python
import requests

class WeatherError(Exception):
    """Raised when there is an error fetching weather data."""
    pass

def get_weather(place):
    """
    Fetches the current weather for a given location using the wttr.in API.

    Args:
        place (str): The location for which to fetch the weather.

    Returns:
        str: The weather description.

    Raises:
        WeatherError: If the request to wttr.in fails.
    """
    url = f"http://wttr.in/{place}?format=(%l) %C and %t (feels %f) with winds at %w"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.text
    except requests.exceptions.RequestException as e:
        raise WeatherError(f"Error fetching weather for {place}: {e}") from e
```


In [7]:
task = "Generate a function specification for the code you just wrote."

In [8]:
messages.append({"role": "user", "content": task})

In [9]:
streaming_response = toolkit.model.get_response(model, messages)
specification_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": specification_response})

```json
{
  "name": "get_weather",
  "description": "Fetches the current weather for a given location using the wttr.in API. Returns a string containing the weather description.",
  "parameters": {
    "type": "object",
    "properties": {
      "place": {
        "type": "string",
        "description": "The location for which to fetch the weather (e.g., 'London', 'New York')."
      }
    },
    "required": ["place"]
  },
  "responses": [
    {
      "type": "string",
      "description": "A string containing the weather description for the given location."
    }
  ],
  "errors": [
    {
      "name": "WeatherError",
      "description": "Raised when there is an error fetching weather data from the wttr.in API (e.g., network error, invalid location)."
    }
  ]
}
```

In [10]:
function = toolkit.functions.parse_funcs(function_response)[0]
specification = toolkit.functions.parse_specs(specification_response)[0]

function = await toolkit.code.define_function(specification["name"], function)
toolkit.functions.register(function, specification)

In [11]:
prompt = toolkit.prompts.function_calling(model)
task = "What's the weather in Pune"

In [12]:
messages.append({"role": "user", "content": prompt})
messages.append({"role": "user", "content": task})

In [13]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
Okay, the user wants to know the weather in Pune. I have a `get_weather` function that can fetch this information. I will call this function with the location "Pune" and return the result to the user.

Guidelines followed: Identified the user's need, identified the relevant function, and prepared to call it.
Guidelines not followed: None.
</blockquote>



```json
{
	"id": "1",
	"function": "get_weather",
	"parameters": {
		"place": "Pune"
	}
}
```

In [14]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
render_markdown(outputs)

```json
{
    "id": "1",
    "result": "(Pune) Patchy rain nearby and +26\u00b0C (feels +28\u00b0C) with winds at \u219223km/h"
}
```


In [15]:
messages.append({"role": "user", "content": outputs})

In [16]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

The weather in Pune is patchy rain and 26°C (feels like 28°C) with winds at 23km/h. 

<blockquote>
I received the function output with the weather information for Pune. I am now presenting this information to the user in a human-readable format.

Guidelines followed: I successfully processed the function output and presented the result to the user.
Guidelines not followed: None.
</blockquote>



In [17]:
await toolkit.code.cleanup_sandbox()